## Loading the necessary libraries

In [ ]:
# Load libraries
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
tf.random.set_seed(0)

from numpy.random import seed
seed(1)

## Load and get a glimpse of the training and test sets

In [ ]:
# Load training data
train_data = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/train.csv', index_col=0)
len(train_data)

In [ ]:
train_data.head()

In [ ]:
# Load testing data
test_data = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/test.csv', index_col=0)
len(test_data)

In [ ]:
test_data.head()

## Pre-processing of data

In [ ]:
le = LabelEncoder()
train_data['target'] = le.fit_transform(train_data['target'])

In [ ]:
train_data

In [ ]:
X = train_data.iloc[:, :-1].values
y = train_data.iloc[:, -1].values

X.shape, y.shape

## Splitting training and test data

In [ ]:
#Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.2, stratify=y, shuffle=True)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
sc = MinMaxScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

testData = sc.transform(test_data.values)

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes=len(np.unique(y)))
y_test = tf.keras.utils.to_categorical(y_test, num_classes=len(np.unique(y)))

In [ ]:
y_test

## Model architecture definition, compilation and training

In [ ]:
from tensorflow.keras.activations import swish
from tensorflow.keras.models import Sequential

model = Sequential([layers.Dense(units=128, activation=swish, input_shape=[X.shape[1]]),
                    layers.Dense(units=32, activation=swish),
                    layers.Dense(units=16, activation=swish),
                    layers.Dense(units=8, activation=swish),
                    layers.Dense(units=9, activation='softmax')])

In [ ]:
es = early_stopping = EarlyStopping(monitor='val_loss', patience=10, min_delta=0.001, restore_best_weights=True, verbose=1)
lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_delta=0.0002, verbose=1)

In [ ]:
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy')

In [ ]:
%%time
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    batch_size=32,
                    epochs=200,
                    callbacks=[es, lr])

In [ ]:
df = pd.DataFrame(history.history)
df[['val_loss', 'loss']].plot()

## Obtaining the test predictions

In [ ]:
test_preds = model.predict(testData)
print(test_preds[0])

## Preparing the format for submission

In [ ]:
# The lines below shows how to save predictions in format used for competition scoring.
output = pd.DataFrame({'id': test_data.index,
                       'Class_1': test_preds[:, 0],
                       'Class_2': test_preds[:, 1],
                       'Class_3': test_preds[:, 2],
                       'Class_4': test_preds[:, 3],
                       'Class_5': test_preds[:, 4],
                       'Class_6': test_preds[:, 5],
                       'Class_7': test_preds[:, 6],
                       'Class_8': test_preds[:, 7],
                       'Class_9': test_preds[:, 8],
                      })

output.to_csv('submission_tps.csv', index=False)

Visualizing the submission file below:

In [ ]:
output.head()